## Task 3  Pizza Place Sales

In [237]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

In [247]:
# Load datasets (handle encoding issues)
order_details = pd.read_csv(r"C:\Users\timi\Downloads\pizza_sales\order_details.csv")
orders = pd.read_csv(r"C:\Users\timi\Downloads\pizza_sales\orders.csv")
pizzas = pd.read_csv(r"C:\Users\timi\Downloads\pizza_sales\pizzas.csv", e)
pizza_types = pd.read_csv(r"C:\Users\timi\Downloads\pizza_sales\pizza_types.csv", encoding='latin1')

# Preview data
orders.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x91 in position 1710: invalid start byte

In [ ]:
print("Orders:", orders.shape)
print("Order Details:", order_details.shape)
print("Pizzas:", pizzas.shape)
print("Pizza Types:", pizza_types.shape)


### Data Cleaning

In [ ]:
# Check for missing values
print("Missing values in Orders:\n", orders.isnull().sum())
print("\nMissing values in Order Details:\n", order_details.isnull().sum())
print("\nMissing values in Pizzas:\n", pizzas.isnull().sum())
print("\nMissing values in Pizza Types:\n", pizza_types.isnull().sum())



In [ ]:
# Check for duplicates
print("Orders duplicates:", orders.duplicated().sum())
print("Order Details duplicates:", order_details.duplicated().sum())
print("Pizzas duplicates:", pizzas.duplicated().sum())
print("Pizza Types duplicates:", pizza_types.duplicated().sum())


In [ ]:
# Convert date column to datetime
orders['date'] = pd.to_datetime(orders['date'])

# Extract useful date features
orders['day'] = orders['date'].dt.day_name()
orders['month'] = orders['date'].dt.month_name()
orders['hour'] = pd.to_datetime(
orders['time'], format='%H:%M:%S').dt.hour


### Cleaned the data by checking for missing values, removing duplicates, and converting date columns into proper datetime format to enable time-based analysis.

In [ ]:
orders['date'] = pd.to_datetime(orders['date'])
order_details['quantity'] = order_details['quantity'].astype(int)
pizzas['price'] = pizzas['price'].astype(float)


In [ ]:
orders.columns = orders.columns.str.lower().str.strip()
order_details.columns = order_details.columns.str.lower().str.strip()
pizzas.columns = pizzas.columns.str.lower().str.strip()
pizza_types.columns = pizza_types.columns.str.lower().str.strip()


In [ ]:
# Merge all four tables
df = order_details.merge(orders, on='order_id') \
                  .merge(pizzas, on='pizza_id') \
                  .merge(pizza_types, on='pizza_type_id')

df.head()


In [ ]:
# Extract additional features
df['day_of_week'] = df['date'].dt.day_name()
df['month'] = df['date'].dt.month_name()



## Analysis


In [ ]:
# Create revenue column
df['revenue'] = df['quantity'] * df['price']

df.head()

In [ ]:
## What is the total revenue/sales?

total_revenue = df['revenue'].sum()
print(f"The total revenue is ${total_revenue:,.2f}")


In [ ]:
## Find the total quantity sold.

total_quantity = df['quantity'].sum()
print(f"The total quantity sold is {total_quantity:,} pizzas")


In [ ]:
## Find the total orders.

total_orders = df['order_id'].nunique()
print(f"The total number of orders is {total_orders:,}")



In [ ]:
## How many pizza types do they sell?

pizza_types_count = df['name'].nunique()
print(f"The total number of pizza types sold is {pizza_types_count}")


In [ ]:
## Find the average price of the pizzas.


In [ ]:
##What are the peak hours of sales?
hourly_sales = df.groupby('hour')['revenue'].sum()

peak_hour = hourly_sales.idxmax()

print(f"The peak hour of sales is {peak_hour}:00")




In [ ]:
plt.figure()
hourly_sales.plot(kind='bar')
plt.title("Revenue by Hour")
plt.xlabel("Hour")
plt.ylabel("Revenue")
plt.show()


### The highest revenue is generated during lunch hours, especially around midday.

In [ ]:
## Find the total sales made on each day of the week. 
sales_by_day = df.groupby('day_of_week')['revenue'].sum().sort_values(ascending=True)

day_order = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday",
               "Saturday"]

sales_by_day = sales_by_day.reindex(day_order)
sales_table = sales_by_day.reset_index()
sales_table.columns = ['Day of Week', 'Total Sales']

print(f"The total sales for each day is in the table below")
display(sales_table)

## Which day of the week is when sales are made the most?
best_day = sales_by_day.idxmax()
print(f"The day with the highest sales is {best_day}.")


In [ ]:
plt.figure()
sales_by_day.plot(kind='bar')
plt.title("Revenue by Day of Week")
plt.xlabel("Day")
plt.ylabel("Revenue")
plt.show()

### Sales are highest on Fridays, indicating strong pre-weekend demand.

In [ ]:
## Find the top 5 bestselling pizzas.
top_5 = df.groupby('name')['quantity'].sum().sort_values(ascending=False).head(5)

top_5_table = top_5.reset_index()
top_5_table.columns = ['Name', 'Quantity']

print("The top 5 bestselling pizzas are:")
display(top_5_table)


In [ ]:
plt.figure()
top_5.plot(kind='bar')
plt.title("Top 5 Bestselling Pizzas")
plt.xlabel("Pizza Name")
plt.ylabel("Quantity Sold")
plt.xticks(rotation=45)
plt.show()


In [ ]:
## Find the sales made in each month
sales_by_month = df.groupby('month')['revenue'].sum().sort_values(ascending=True)

month_order = ["January", "February", "March", "April", "May", "June",
               "July", "August", "September", "October", "November", "December"]

sales_by_month = sales_by_month.reindex(month_order)

month_order_table = sales_by_month.reset_index()
month_order_table.columns = ['Month', 'Sales']

print(f"The sales made in each month is listed below")
display(month_order_table)

best_month = sales_by_month.idxmax()
print(f"The month with the highest sales is {best_month}")


In [ ]:
plt.figure()
sales_by_month.plot(kind='bar')
plt.title("Revenue by Month")
plt.xlabel("Month")
plt.ylabel("Revenue")
plt.xticks(rotation=45)
plt.show()


### Any trend noticeable? - Sales show seasonal fluctuations, with noticeable peaks during mid-year months.

In [ ]:
bottom_5 = df.groupby('name')['quantity'].sum().sort_values(ascending=True).head(5)


bottom_5_table = bottom_5.reset_index()
bottom_5_table.columns = ['Name', 'Quantity Sold']


print("The lowest performing pizzas are:")
display(bottom_5_table)

In [ ]:
plt.figure()
bottom_5.plot(kind='bar')
plt.title("Bottom 5 Performing Pizzas")
plt.xlabel("Pizza Name")
plt.ylabel("Quantity Sold")
plt.xticks(rotation=45)
plt.show()


#### Some pizzas sell significantly less than others and may require marketing promotion or menu review.

#### The pizza business performs strongest during lunch hours and Fridays.
#### Certain pizza types dominate sales, while a few underperform and may need strategic attention.
#### Overall revenue remains stable throughout the year with slight seasonal variation.